In [3]:
!apt install unzip


Reading package lists... Done
Building dependency tree       
Reading state information... Done
unzip is already the newest version (6.0-21ubuntu1.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


In [4]:
!pip install --upgrade --force-reinstall --no-deps kaggle


Processing /root/.cache/pip/wheels/a1/6a/26/d30b7499ff85a4a4593377a87ecf55f7d08af42f0de9b60303/kaggle-1.5.12-cp37-none-any.whl
  Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12


In [5]:
from google.colab import files
files.upload()


Saving kaggle.json to kaggle (1).json


{'kaggle.json': b'{"username":"dido1912","key":"043008531b71256eb84d955b134ec8ff"}'}

In [6]:
! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/

! chmod 600 ~/.kaggle/kaggle.json
! kaggle competitions download -c 'uos-com2028'


mkdir: cannot create directory ‘/root/.kaggle’: File exists
uos-com2028.zip: Skipping, found more recently modified local copy (use --force to force download)


In [7]:
! unzip uos-com2028.zip -d AI_Coursework

Archive:  uos-com2028.zip
replace AI_Coursework/submission_sample.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [8]:
#Imports
import keras
import numpy as np
from numpy.random import seed
import tensorflow as tf
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import os
import cv2
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from cv2 import cv2
from tqdm import tqdm
%matplotlib inline


In [9]:
#Upload the data and make an array for the training images
trained_images = [None] * 10270
savedIn = 'AI_Coursework/train/train'
for i in tqdm(os.listdir(savedIn)):
  path = os.path.join(savedIn,i)
  img = cv2.imread(path,cv2.IMREAD_COLOR)
  img = cv2.resize(img, (64,64))
  img = img/255
  i = i[:-4]
  trained_images[int(i)] = np.array(img)
x_train = np.array(trained_images)


100%|██████████| 10270/10270 [00:42<00:00, 240.75it/s]


In [10]:
#Checking if the shape is okay
x_train.shape


(10270, 64, 64, 3)

In [11]:
#Upload the data annd make an array for the testing images
tested_images = [None] * 15009
savedIn = 'AI_Coursework/test/test'
for i in tqdm(os.listdir(savedIn)):
  path = os.path.join(savedIn,i)
  img = cv2.imread(path,cv2.IMREAD_COLOR)
  img = cv2.resize(img, (64,64))
  img = img/255
  i = i[:-4]
  tested_images[int(i)] = np.array(img)
x_test = np.array(tested_images)


100%|██████████| 15009/15009 [01:02<00:00, 240.27it/s]


In [12]:
#Check if the shape of x_test is okay
x_test.shape


(15009, 64, 64, 3)

In [13]:
#Get the Y
example = "AI_Coursework/train.txt"
file = open(example, "r")
lines = file.readlines()
target = []
for line in lines:
  for n in line.split()[1:]:
    target.append(int(n))
y_train = np.array(target)
print(y_train)


[ 5 21 18 ...  7  7  7]


In [14]:
from keras. utils import to_categorical
y_train = to_categorical(y_train)
y_train.shape


(10270, 23)

In [15]:
X_train, X_test, Y_train, Y_test = train_test_split(x_train, y_train)


In [16]:
from keras.models import Sequential
#Adding dense layer imports
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils

model = Sequential()
model.add(Convolution2D(64, kernel_size = 3, activation='relu', input_shape=(64,64,3)))
#model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Convolution2D(64, kernel_size = 3, activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Dropout(0.2)) # could be changed to 0.25

model.add(Convolution2D(64, kernel_size = 3, activation='relu'))
model.add(Convolution2D(64, kernel_size = 3, activation='relu')) # could have it 2 no problem
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Dropout(0.3))

model.add(Convolution2D(128, kernel_size = 3, activation='relu'))
model.add(Dropout(0.5))

model.add(Flatten())

#23 because of the 23 classes of butterflies available; could use either softmax or relu
model.add(Dense(23, activation='softmax')) 

# Use of 'adam' to optimize, as it is the best practice considered
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

#Compile the model
model.compile(optimizer='adam', loss = 'categorical_crossentropy', metrics=['accuracy'])

print(model.summary()) #print statement could be omitted


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 64)        1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 60, 60, 64)        36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 20, 20, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 20, 20, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 18, 18, 64)        36928     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0

In [17]:
# Fit the model
model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=100, batch_size=258)


Epoch 1/100
30/30 [==============================] - 41s 295ms/step - loss: 2.8659 - accuracy: 0.1428 - val_loss: 2.6546 - val_accuracy: 0.2504
Epoch 2/100
30/30 [==============================] - 5s 172ms/step - loss: 2.5685 - accuracy: 0.2539 - val_loss: 2.3785 - val_accuracy: 0.2862
Epoch 3/100
30/30 [==============================] - 5s 172ms/step - loss: 2.3875 - accuracy: 0.2980 - val_loss: 2.2890 - val_accuracy: 0.3111
Epoch 4/100
30/30 [==============================] - 5s 172ms/step - loss: 2.2687 - accuracy: 0.3204 - val_loss: 2.1286 - val_accuracy: 0.3489
Epoch 5/100
30/30 [==============================] - 5s 174ms/step - loss: 2.1631 - accuracy: 0.3567 - val_loss: 2.0878 - val_accuracy: 0.3641
Epoch 6/100
30/30 [==============================] - 5s 172ms/step - loss: 2.0662 - accuracy: 0.3852 - val_loss: 1.9708 - val_accuracy: 0.4065
Epoch 7/100
30/30 [==============================] - 5s 174ms/step - loss: 1.9854 - accuracy: 0.4027 - val_loss: 1.9231 - val_accuracy: 0.417

In [18]:
loss_and_metrics = model.evaluate(X_test, Y_test, verbose=2)
print("Test Loss", loss_and_metrics[0])
print("Accuracy of Model", loss_and_metrics[1]*100, '%')


81/81 - 1s - loss: 1.3553 - accuracy: 0.6655
Test Loss 1.3553181886672974
Accuracy of Model 66.5498435497284 %


In [19]:
#Make the y_pred
y_pred = model.predict(x_test)
y_pred = np.argmax(y_pred, axis=1)

print(len(y_pred))


15009


In [21]:
#Creating ther .csv file
col = pd.read_csv("AI_Coursework/submission_sample.csv")
col["label"] = y_pred
col.to_csv("AI_Coursework/dk00504_CW.csv", index=False)
